In [1]:
### generate complex data 
import numpy as np
np.random.seed(5)

def generate_data(n_samples, true_coefficients, noise_level=0.1):

    # Generate a random input signal
    k = len(true_coefficients)
    m, n = true_coefficients[0].shape


    x = np.random.randn(n_samples, m) + 1j*np.random.randn(n_samples, m)

    # Generate the desired signal by passing the input through the known system
    y = np.zeros((n_samples, n), dtype=np.complex128)

    ### y is complex data
    for j in range(k):
        y += np.dot(np.roll(x, j, axis=0), true_coefficients[j])

    # Add noise
    noise = noise_level * np.random.randn(n_samples, n) + 1j*noise_level * np.random.randn(n_samples, n)
    y += noise
    
    return x, y

# Example usage
n_samples = 100000
# a series of matrix: k x m x n 
# k : influence scope 5
# m : output dimension  3
# n : input dimension 2

k = 2
m = 3
n = 2
## coef is complex 
true_coefficients = [np.random.randn(m, n) + 1j*np.random.randn(m, n) for _ in range(k)]

noise_level = 0.05
x, y = generate_data(n_samples, true_coefficients, noise_level)
print("Input signal:", x[:3])
print("Desired signal:", y[:3])

Input signal: [[ 0.99643983+0.44662929j  0.71242127-0.2265968j   0.05914424-0.96532997j]
 [-0.36331088-1.41035519j  0.00328884+0.53010945j -0.10593044+1.85599642j]
 [ 0.79305332+2.11510082j -0.63157163+0.51149553j -0.00619491-1.03972511j]]
Desired signal: [[-1.9499539 -3.47171794j -1.31378069-1.51771071j]
 [-1.2222745 -1.7343442j  -0.98263331+2.10901126j]
 [-0.61794525+3.63097484j  2.80727439-0.67160134j]]


In [2]:
### LMS filter
k = len(true_coefficients)
def lms_filter(x, y, k, mu ):
    Ws = [] 
    m, n = x.shape[1], y.shape[1]
    w = np.array([np.random.randn(m, n) + 1j*np.random.randn(m, n) for _ in range(k)]) ## k x m x n
    e = np.zeros((len(x), n), dtype=complex) ## record error

    for i in range(k, len(x)):
        y_hat = np.zeros((n), dtype=complex)
        for j in range(k):
            y_hat += np.dot(x[i-j], w[j])
        error = y[i] - y_hat 
        e[i] = error
        for j in range(k):
            ### update w[j], using wirtinger derivative
            w[j] += mu * np.outer(np.conj(x[i-j]) , error)
        Ws.append(w.reshape(k, m, n).copy())
    return w,e, Ws


In [3]:
w,e, Ws = lms_filter(x, y, k, 0.01)
print("Estimated coefficients:", w[1])
print ("True coefficients:", true_coefficients[1])

Estimated coefficients: [[-0.36436974-1.51473762j  0.60379671+0.64909497j]
 [-1.66129671-0.989564j   -0.7028862 -0.85923641j]
 [ 1.15555508-0.8717776j   1.86323032-0.42082512j]]
True coefficients: [[-0.35882895-1.51117956j  0.6034716 +0.64484751j]
 [-1.66478853-0.98060789j -0.70017904-0.85685315j]
 [ 1.15139101-0.87187918j  1.85733101-0.42250793j]]


In [4]:
y[4]

array([ 1.1899848 +2.91902382j, -0.06707877-5.32295825j])

In [5]:
print( np.dot(x[4], true_coefficients[0]) ) 

[2.33657787+4.0258594j  0.1197953 -2.83836461j]


In [6]:
Ws[300]

array([[[ 0.42418699-0.91273998j, -0.32888752-0.6002173j ],
        [ 2.41559575+0.19225603j, -0.25167144-0.32445637j],
        [ 0.11309557-1.19387541j,  1.57072339-0.20108114j]],

       [[-0.34294035-1.50696159j,  0.60178546+0.63642686j],
        [-1.6606238 -0.9793118j , -0.69986166-0.86560641j],
        [ 1.15385128-0.86231348j,  1.85149226-0.42265886j]]])

In [7]:
true_coefficients[0].shape

(3, 2)